In [1]:
import warnings
warnings.filterwarnings('ignore')
import pickle
import cv2
import numpy as np

In [3]:
# pip install -qr https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt

### Load Pretrained Models for Distance Estimation

In [33]:
import pickle
import os 
from tensorflow import keras
model_dir = "model"
model_name = 'LinearRegression'
model_path = os.path.join(model_dir, model_name)
pkl_filename = model_path + "/mlr_model.pkl"

with open(pkl_filename, 'rb') as file:
    mlr_model = pickle.load(file)
    print("Linear Regression Model Loaded Successfully")

Linear Regression Model Loaded Successfully


In [42]:

model_dir = "model"
model_name = 'ridge'
model_path = os.path.join(model_dir, model_name)
pkl_filename = model_path + "/ridge_model.pkl"

with open(pkl_filename, 'rb') as file:
    ridge_model = pickle.load(file)
    print("Ridge Regression Model Loaded Successfully")

Ridge Regression Model Loaded Successfully


In [35]:
model_dir = "model"
model_name = 'Lasso'
model_path = os.path.join(model_dir, model_name)
pkl_filename = model_path + "/lasso_model.pkl"

with open(pkl_filename, 'rb') as file:
    lasso_model = pickle.load(file)
    print("Lasso Regression Model Loaded Successfully")

Lasso Regression Model Loaded Successfully


In [36]:
model_dir = "model"
model_name = 'Knn'
model_path = os.path.join(model_dir, model_name)
pkl_filename = model_path + "/knn_model.pkl"

with open(pkl_filename, 'rb') as file:
    knn_model = pickle.load(file)
    print("Knn Regression Model Loaded Successfully")

Knn Regression Model Loaded Successfully


In [37]:
model_dir = "model"
model_name = 'SVM'
model_path = os.path.join(model_dir, model_name)
pkl_filename = model_path + "/svm.pkl"

with open(pkl_filename, 'rb') as file:
    SVM = pickle.load(file)
    print("Support Vector Regression Model Loaded Successfully")

Support Vector Regression Model Loaded Successfully


In [38]:
model_dir = "model"
model_name = 'DecisionTree'
model_path = os.path.join(model_dir, model_name)
pkl_filename = model_path + "/DT_regressor.pkl"

with open(pkl_filename, 'rb') as file:
    DT_regressor = pickle.load(file)
    print("Decision Tree Regression Model Loaded Successfully")

Decision Tree Regression Model Loaded Successfully


In [39]:
model_dir = "model"
model_name = 'Randomforest'
model_path = os.path.join(model_dir, model_name)
pkl_filename = model_path + "/rf_model.pkl"

with open(pkl_filename, 'rb') as file:
    rf_model = pickle.load(file)
    print("Random Forest Regression Model Loaded Successfully")

Random Forest Regression Model Loaded Successfully


In [40]:

SNN = keras.models.load_model('model/SNN/SNN.pb')

print("Simple Neural Network Regression Model Loaded Successfully")

Simple Neural Network Regression Model Loaded Successfully


In [41]:
LSTM = keras.models.load_model('model/LSTM/lstm.h5')

print("LSTM Regression Model Loaded Successfully")

LSTM Regression Model Loaded Successfully


### Load the Pretrained Yolov5 Object Detection Model

In [74]:
import torch

# Model
yolov5model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# Images
imgs = ['test_images/person_278.jpeg']  # batch of images
image_name = 'test_images/person_278.jpeg'
# Inference
results = yolov5model(imgs)

# Results
results.print()
results.save()  # or .show()

results.xyxy[0]  # img1 predictions (tensor)
results.pandas().xyxy[0]

Using cache found in C:\Users\ASUS/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5 🚀 2022-8-13 Python-3.9.7 torch-1.12.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 
Saved 1 image to runs\detect\exp38


image 1/1: 1280x960 1 person, 1 bench
Speed: 26.0ms pre-process, 194.0ms inference, 1.0ms NMS per image at shape (1, 3, 640, 480)


,xmin,ymin,xmax,ymax,confidence,class,name
0,367.494202,267.399109,621.922668,567.702576,0.853258,0,person
1,0.000000,526.482788,926.175171,1273.121216,0.414770,13,bench


In [75]:
df = results.pandas().xyxy[0]

In [76]:
df.head()

,xmin,ymin,xmax,ymax,confidence,class,name
0,367.494202,267.399109,621.922668,567.702576,0.853258,0,person
1,0.000000,526.482788,926.175171,1273.121216,0.414770,13,bench


In [77]:
eachrow = list()
for i in range(df.shape[0]):
    eachrow.append(list(df.loc[i]))
print(eachrow)

[[367.49420166015625, 267.39910888671875, 621.9226684570312, 567.7025756835938, 0.8532581925392151, 0, 'person'], [0.0, 526.4827880859375, 926.1751708984375, 1273.1212158203125, 0.4147697985172272, 13, 'bench']]


In [78]:
box = list()
for i in eachrow:
    box.append(i[:6])
box

[[367.49420166015625,
  267.39910888671875,
  621.9226684570312,
  567.7025756835938,
  0.8532581925392151,
  0],
 [0.0,
  526.4827880859375,
  926.1751708984375,
  1273.1212158203125,
  0.4147697985172272,
  13]]

### Resize Image to original

In [79]:
# import numpy as np

# top, left, bottom, right = box
# top = max(0, np.floor(top + 0.5).astype('int32'))
# left = max(0, np.floor(left + 0.5).astype('int32'))
# bottom = min(image.size[1], np.floor(bottom + 0.5).astype('int32'))
# right = min(image.size[0], np.floor(right + 0.5).astype('int32'))

### Get image size

In [80]:
from PIL import Image

imagetest = Image.open(image_name)
img_width, img_height = imagetest.size
print(imagetest.size)


(960, 1280)


### create the input for models

In [82]:
model_input = list()
zipped_list = list()
for i in range(len(box)):

    temp_input = list()
    
    top, left, bottom, right = box[i][:4]
    top = max(0, np.floor(top + 0.5).astype('int32'))
    left = max(0, np.floor(left + 0.5).astype('int32'))
    bottom = min(imagetest.size[1], np.floor(bottom + 0.5).astype('int32'))
    right = min(imagetest.size[0], np.floor(right + 0.5).astype('int32'))
    width = float(right - left) / img_width
    height = float(bottom - top) / img_height
    diagonal = np.sqrt(np.square(width) + np.square(height))
    
    if box[i][5] == 0:
        class_h, class_w, class_d = np.array([175, 55, 30], dtype=np.float32)
    elif box[i][5] == 1:
        class_h, class_w, class_d = np.array([120, 30, 170], dtype=np.float32)
    elif box[i][5] == 2:                  
        class_h, class_w, class_d = np.array([160, 180, 400], dtype=np.float32)
        
    temp_input = [1 / width, 1 / height, 1 / diagonal, class_h, class_w, class_d]
    model_input.append(np.array(temp_input))
    
    index_name = ['width', 'height', 'diagonal', 'size_h', 'size_w', 'size_d']
    zipped_list.append(list(zip(index_name,temp_input)))
    
print(model_input)
print("*************************")
print(zipped_list)

[array([     3.1894,      5.0196,      2.6919,         175,          55,          30]), array([      2.212,      1.3823,      1.1722,         175,          55,          30])]
*************************
[[('width', 3.1893687707641196), ('height', 5.019607843137255), ('diagonal', 2.691943773416355), ('size_h', 175.0), ('size_w', 55.0), ('size_d', 30.0)], [('width', 2.2119815668202767), ('height', 1.3822894168466524), ('diagonal', 1.1722264907854945), ('size_h', 175.0), ('size_w', 55.0), ('size_d', 30.0)]]


In [83]:
# for i in range(len(box)):
#     top, left, bottom, right = box[i]

#     width = float(right - left) / img_width
#     height = float(bottom - top) / img_height
#     diagonal = np.sqrt(np.square(width) + np.square(height))
#     class_h, class_w, class_d = np.array([175, 55, 30], dtype=np.float32)
#     model_input = [1 / width, 1 / height, 1 / diagonal, class_h, class_w, class_d]
#     index_name = ['width', 'height', 'diagonal', 'size_h', 'size_w', 'size_d']
#     zipped_list = list(zip(index_name,dist_input))
#     model_input = np.array(dist_input)
#     print(model_input)
#     print(zipped_list)

In [84]:
test_input = list()
test_input = np.array(model_input)
print(test_input)
print(test_input.shape)

[[     3.1894      5.0196      2.6919         175          55          30]
 [      2.212      1.3823      1.1722         175          55          30]]
(2, 6)


## Get Distance Prediction using Loaded Models

In [85]:
#Linear Regression
mlr_distance = mlr_model.predict(test_input)
mlr_distance = list(mlr_distance)
print("Linear Regression")
print(mlr_distance,"meter")


# Ridge Regression
ridge_distance = ridge_model.predict(test_input)
ridge_distance = list(ridge_distance)
print("Ridge Regression")
print(ridge_distance,"meter")


# Lasso Regression
lasso_distance = lasso_model.predict(test_input)
lasso_distance = list(lasso_distance)
print("Lasso Regression")
print(lasso_distance,"meter")


# KNN
knn_distance = knn_model.predict(test_input)
knn_distance = list(knn_distance)
print("Knn")
print(knn_distance,"meter")


# SVM
SVM_Distance = SVM.predict(test_input)
SVM_Distance = list(SVM_Distance)
print("SVM")
print(SVM_Distance,"meter")


# Decision Tree
DT_Distance = DT_regressor.predict(test_input)
DT_Distance = list(DT_Distance)
print("Decision Tree")
print(DT_Distance,"meter")


# Random Forest
rf_Distance = rf_model.predict(test_input)
rf_Distance = list(rf_Distance)
print("Random Forest")
print(rf_Distance,"meter")


# SNN
SNN_Distance = SNN.predict(test_input)
temp_snn = []
for i in SNN_Distance:
    temp_snn.extend(i)
SNN_Distance = temp_snn
print("Simple Neural Network")
print(SNN_Distance,"meter")


# LSTM
LSTM_Distance = LSTM.predict(test_input)
temp_lstm = []
for j in LSTM_Distance:
    temp_lstm.extend(j)
LSTM_Distance = temp_lstm
print("Long term Short Term")
print(LSTM_Distance,"meter")

Linear Regression
[6.708183001194545, 4.334927970283264] meter
Ridge Regression
[6.708384025016809, 4.3350749786526315] meter
Lasso Regression
[6.767375577638166, 4.388515052491762] meter
Knn
[6.2521758964447525, 3.1937053626424516] meter
SVM
[5.5913147881255725, 2.2113084510616403] meter
Decision Tree
[6.72434584217368, 2.5351614626971157] meter
Random Forest
[7.667660862914906, 2.068088338175786] meter
Simple Neural Network
[8.056522, 2.496179] meter
Long term Short Term
[8.475243, 3.358922] meter


In [86]:
for box in results.xyxy[0]:
    print(box)

tensor([367.49420, 267.39911, 621.92267, 567.70258,   0.85326,   0.00000])
tensor([0.00000e+00, 5.26483e+02, 9.26175e+02, 1.27312e+03, 4.14770e-01, 1.30000e+01])


In [88]:
import cv2
index = 0
frame = cv2.imread(image_name)

for box in results.xyxy[0]:

    if box[5]==0:
        xB = int(box[2])
        xA = int(box[0])
        yB = int(box[3])
        yA = int(box[1])
        rect1 = cv2.rectangle(frame, (xA, yA), (xB, yB), (0, 255, 0), 1)
        cv2.putText(rect1, str(SVM_Distance[index])+'meter', (xA,yA-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (36,255,12), 2)
        
    if box[5]==1:
        xB = int(box[2])
        xA = int(box[0])
        yB = int(box[3])
        yA = int(box[1])
        rect2 = cv2.rectangle(frame, (xA, yA), (xB, yB), (0, 255, 0), 1)
        cv2.putText(rect2, str(SVM_Distance[index]), (xA,yA-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (36,255,12), 1)
        
    if box[5]==2:
        xB = int(box[2])
        xA = int(box[0])
        yB = int(box[3])
        yA = int(box[1])
        rect2 = cv2.rectangle(frame, (xA, yA), (xB, yB), (0, 255, 0), 1)
        cv2.putText(rect2, str(SVM_Distance[index]), (xA,yA-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (36,255,12), 1)
    index = index + 1
    
cv2.imshow("OutputWindow",frame)
# cv2.imwrite(""output_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [25]:
cv2.imwrite("output1.png",frame)

True